# curating pr data

# Import general packages
- if it is the first time running this notebook, will need to set up environment ->
locally I'm just using my stitches interpreter

In [ ]:
import stitches as stitches


import pandas as pd
import pkg_resources
import xarray as xr
import numpy as np
import seaborn as sns

# Plotting options
sns.set(font_scale=1.3)
sns.set_style("white")
# For help with plotting
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6
pd.set_option('display.max_columns', None)

# import packages for spatial masking

In [ ]:
import geopandas as gpd
# Spatial subsetting of netcdf files:
import regionmask

#  Set up time slices and area of interest (AOI) to focus on

- require ensemble average PR values over the ref period and comparison period
for an area of interest
- Do spatial aggregation for each ensemble member, take the time average in the
time window, calculate average across ensemble members

In [ ]:
# Time slices
ref_start = '1995-01-01'
ref_end =  '2014-12-31'

comp_start = '2080-01-01'
comp_end =  '2099-12-31'

window_length = 20

In [ ]:
# AOI
# working off https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/subset-netcdf4-climate-data-spatially-aoi/

# # physical land polygon files:
# url =  (    "https://naturalearth.s3.amazonaws.com/"
# "10m_physical/ne_10m_land.zip")

# # country URL
# url =  (    "https://naturalearth.s3.amazonaws.com/"
#             "10m_cultural/ne_10m_admin_0_countries.zip")

# state/province URL
url =  (    "https://naturalearth.s3.amazonaws.com/"
            "10m_cultural/ne_10m_admin_1_states_provinces.zip")


land_main_gdf = gpd.read_file(url)
land_main_gdf.head()

In [ ]:
# define conus as AOI
aoi = land_main_gdf[(land_main_gdf['admin'] == 'United States of America')&
                    (land_main_gdf['name'] != 'Hawaii') &
                     (land_main_gdf['name'] != 'Alaska') ].reset_index(drop=True).copy()

print(aoi.total_bounds)
# Get lat min, max
aoi_lat = [float(aoi.total_bounds[1]), float(aoi.total_bounds[3])]
aoi_lon = [float(aoi.total_bounds[0]), float(aoi.total_bounds[2])]
# The netcdf files use a global lat/lon so adjust values accordingly
aoi_lon[0] = aoi_lon[0] + 360
aoi_lon[1] = aoi_lon[1] + 360

aoi_lon, aoi_lat

# specify ESMs, variables, experiments

In [ ]:
# The CMIP6 ESM we want to emulate and the variables we want to
# emulate


esm = ['CAMS-CSM1-0', 'MIROC6', 'GFDL-ESM4', 'FGOALS-g3',
'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
'ACCESS-ESM1-5', 'IPSL-CM6A-LR', 'CESM2-WACCM',
#'UKESM1-0-LL',
'CanESM5']
vars1 = ['pr']

exps = ['historical','ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp460', 'ssp585',
        'ssp434', 'ssp534-over']

# Pull pangeo dataframe with netcdf addresses for above

In [ ]:
# pangeo table of ESMs for reference
pangeo_path = pkg_resources.resource_filename('stitches', 'data/pangeo_table.csv')
pangeo_data = pd.read_csv(pangeo_path)
# print(np.sort(pangeo_data.variable.unique()))
pangeo_data = pangeo_data[((
                               (pangeo_data['variable'].isin(vars1)) )  )
                          & ((pangeo_data['domain'].str.contains('mon')) ) &
                           ((pangeo_data['experiment'].isin(exps))) &
                           (pangeo_data['model'].isin(esm))].copy()

print(pangeo_data.head())


# loop over files and do calculations

In [ ]:
pr_holder = pd.DataFrame()

for esmname in esm[0:1]:
  for exp in exps[0:5]:

    print(esmname)
    print(exp)
    df_ens_avg = 0

    filelist = pangeo_data[(pangeo_data['model'] ==esmname) & (pangeo_data['experiment'] == exp)].copy()

    # For all ensemble members in this experiment:
    if not filelist.empty:
      df_sum = 0
      n_good_files = 0
      # for each ensemble netcdf:
      for i in range(len(filelist)):
        print(i)

        # Load data and mask to aoi:
        x = stitches.fx_pangeo.fetch_nc(filelist.iloc[i].zstore)
        if (i==0):
            aoi_mask = regionmask.mask_3D_geopandas(aoi,
                                                    x.lon,
                                                    x.lat)
        # # for testing if masking is happening right
        # x.sel(time=slice(ref_start, '1995-01-31')).pr.to_dataframe().reset_index().to_csv('test2.csv', index=False)
        #
        # x = x.sel(time=slice(ref_start, '1995-01-31'),
        # lon = slice(aoi_lon[0], aoi_lon[1]),
        # lat = slice(aoi_lat[0], aoi_lat[1])).where(aoi_mask).copy()
        # x1 = x.pr.to_dataframe().dropna().reset_index().drop(['region'], axis=1).reset_index(drop=True).copy()
        # x1.to_csv('test.csv', index=False)
        x = x.sel(lon = slice(aoi_lon[0], aoi_lon[1]),
                  lat = slice(aoi_lat[0], aoi_lat[1])).where(aoi_mask).copy()

        # If the experiment is historical, further slice to reference years.
        # Otherwise, slice to comparison years:
        if (exp == 'historical'):
            x = x.sel(time=slice(ref_start, ref_end)).copy()
        else:
            x = x.sel(time=slice(comp_start, comp_end)).copy()

        # Check if there are the correct number of time steps in this
        # sliced data:
        # Very rough QC for checking complete netcdfs and assumes
        # comparison window and reference window same length.
        if (len(x.time) >= 12*window_length):
            # coerce to DF so we can properly lat weight to do spatial aggregation:
            x1 = x.pr.to_dataframe().dropna().reset_index().drop(['region'],
                                                                 axis=1).reset_index(drop=True).copy()
            # spatial aggregation:
            monthly_aoi_pr = pd.DataFrame()
            for name, group in x1.groupby('time'):
                lat = group['lat']
                area = np.cos(np.deg2rad(lat))
                df = pd.DataFrame({'time': group['time'].drop_duplicates()})
                df['aggregate_pr'] = sum(area * group['pr'])/sum(area)
                monthly_aoi_pr = pd.concat([df, monthly_aoi_pr]).reset_index(drop=True).copy()
                del(df)
                del(area)
                del(lat)
                # end for loop over months to do spatial disaggregation

            # time average for this ensemble member:
            aoi_pr = monthly_aoi_pr['aggregate_pr'].mean().copy()

            # and add it to the running sum for the ensemble members
            df_sum = (aoi_pr  + df_sum).copy()
            n_good_files = n_good_files + 1
        # end for loop over files in the experiment

    # Calculate the ensemble average of CONUS 20 year average precip for this
    # experiment
    df_ens_avg = df_sum/n_good_files

    # and append to the pr holding data frame
    pr_holder = pd.concat([pr_holder,
                           pd.DataFrame({'esm':esmname,
                                         'experiment':exp,
                                         'ens_avg_pr':df_ens_avg}
                                        )]).reset_index(drop=True).copy()
    del(filelist)
    del(df_ens_avg)
    # end loop over experiments
# end loop over esms


#%